# Objetivo

<span style="color: black; font-family: Arial; font-size: 20px;">
O objetivo deste estudo é analisar dados de um serviço concorrente para entender padrões de publicação, avaliação e engajamento de leitores diante de um cenário onde houveram mudanças de comportamento provocadas pela pandemia do coronavírus onde pessoas passaram a ficar mais em casa e a consumir mais livros.
Essas informações serão fundamentais para embasar decisões estratégicas de desenvolvimento de um novo produto, aumentando as chances de sucesso em um mercado em expansão.

# Carregando os dados

In [4]:
# Importando bibliotecas
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# Configurando as credenciais de conexão
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

# Montando a string de conexão
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Criando a engine
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Fazendo uma consulta de teste e abrindo as primeiras linhas das tabelas
query_books = 'SELECT * FROM books LIMIT 5;'
query_authors = 'SELECT * FROM authors LIMIT 5;'
query_publishers = 'SELECT * FROM publishers LIMIT 5;'
query_ratings = 'SELECT * FROM ratings LIMIT 5;'
query_reviews = 'SELECT * FROM reviews LIMIT 5;'

df_books = pd.read_sql(query_books, con=engine)
df_authors = pd.read_sql(query_authors, con=engine)
df_publishers = pd.read_sql(query_publishers, con=engine)
df_ratings = pd.read_sql(query_ratings, con=engine)
df_reviews = pd.read_sql(query_reviews, con=engine)

print('DF_BOOKS')
print(df_books)
print()
print('DF_AUTHORS')
print(df_authors)
print()
print('DF_PUBLISHERS')
print(df_publishers)
print()
print('DF_RATINGS')
print(df_ratings)
print()
print('DF_REVIEWS')
print(df_reviews)

DF_BOOKS
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

DF_AUTHORS
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        A

# Análises e conclusões

<span style="color: blue; font-family: Arial; font-size: 20px;">
Encontrando o número de livros lançados depois de 1 de janeiro de 2000.

In [11]:
query_1 = """
SELECT COUNT(*) AS num_books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""

books_after_2000 = pd.read_sql(query_1, con=engine)
print(books_after_2000)

   num_books_after_2000
0                   819


<span style="color: black; font-family: Arial; font-size: 20px;">
819 livros foram lançados depois de 1 de janeiro de 2000.

<span style="color: blue; font-family: Arial; font-size: 20px;">
Encontrando o número de avaliações e a classificação média para cada livro.

In [15]:
query_2 = """SELECT
               books.title,
               subquery.avg_rating,
               subquery.review_count
               FROM
                   (SELECT
                        reviews.book_id as book_id,
                        COUNT (DISTINCT reviews.review_id) AS review_count,
                        AVG (ratings.rating) AS avg_rating
                    FROM
                        reviews
                        INNER JOIN ratings ON ratings.book_id = reviews.book_id
                    GROUP BY
                        reviews.book_id
                    ORDER BY
                        review_count DESC,
                        avg_rating DESC) AS subquery
                    INNER JOIN books ON subquery.book_id = books.book_id
               ORDER BY
                   review_count DESC,
                   avg_rating DESC
               LIMIT 10"""

ratings_per_book = pd.read_sql(query_2, con=engine)
print(ratings_per_book.head(10))

                                               title  avg_rating  review_count
0                            Twilight (Twilight  #1)    3.662500             7
1  Harry Potter and the Prisoner of Azkaban (Harr...    4.414634             6
2  Harry Potter and the Chamber of Secrets (Harry...    4.287500             6
3                                     The Book Thief    4.264151             6
4                                   The Glass Castle    4.206897             6
5                The Hobbit  or There and Back Again    4.125000             6
6                          Outlander (Outlander  #1)    4.125000             6
7  The Curious Incident of the Dog in the Night-Time    4.081081             6
8  The Lightning Thief (Percy Jackson and the Oly...    4.080645             6
9                                Water for Elephants    3.977273             6


<span style="color: black; font-family: Arial; font-size: 20px;">
Acima mostrando os 10 primeiros livros com seus números de avaliações e classificações médias.

<span style="color: blue; font-family: Arial; font-size: 20px;">
Identificando a editora que lançou o maior número de livros com mais de 50 páginas.

In [19]:
query_3 = """
SELECT 
    p.publisher_id,
    p.publisher,
    COUNT(b.book_id) AS num_books
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY num_books DESC
LIMIT 1;
"""

best_publisher = pd.read_sql(query_3, con=engine)
print(best_publisher)

   publisher_id      publisher  num_books
0           212  Penguin Books         42


<span style="color: black; font-family: Arial; font-size: 20px;">
'Penguin Books' foi a editora que lançou o maior número de livros com mais de 50 páginas.

<span style="color: blue; font-family: Arial; font-size: 20px;">
Identificando o autor com a média mais alta de classificação de livros (apenas para livros com pelo menos 50 classificações).

In [23]:
query_4 = """SELECT
                   authors.author,
                   AVG (subquery2.avg_rating) as final_avg            
               FROM
                   (SELECT
                       books.title,
                       books.author_id,
                       subquery1.avg_rating
                   FROM
                       (SELECT
                           book_id,
                           COUNT (rating_id) AS rating_cnt,
                           AVG (rating) AS avg_rating
                       FROM
                           ratings
                       GROUP BY
                           book_id
                       HAVING
                           COUNT (rating_id) > 50) AS subquery1
                       INNER JOIN books ON books.book_id = subquery1.book_id) AS subquery2
                   INNER JOIN authors ON authors.author_id = subquery2.author_id
               GROUP BY
                   author
               ORDER BY
                   final_avg DESC
               LIMIT 5"""

best_author = pd.read_sql(query_4, con=engine)
print(best_author)

                              author  final_avg
0         J.K. Rowling/Mary GrandPré   4.283844
1  Markus Zusak/Cao Xuân Việt Khương   4.264151
2                     J.R.R. Tolkien   4.258446
3                  Louisa May Alcott   4.192308
4                       Rick Riordan   4.080645


<span style="color: black; font-family: Arial; font-size: 20px;">
'J.K. Rowling/Mary GrandPré' foi a o autora com a média mais alta de classificação de livros que contém pelo menos 50 classificações, mostrando uma classificação média de 4.28.

<span style="color: blue; font-family: Arial; font-size: 20px;">
Encontrando o número médio de avaliações entre usuários que classificaram mais do que 50 livros.

In [27]:
query_5 = """SELECT
                   AVG (subquery2.review_cnt) AS avg_review_cnt
               FROM
                   (SELECT
                       COUNT (reviews.review_id) as review_cnt,
                       subquery1.username
                   FROM
                       (SELECT
                           username,
                           COUNT (rating_id) AS rating_cnt
                       FROM
                           ratings
                       GROUP BY
                           username
                       HAVING
                           COUNT (rating_id) > 50) AS subquery1
                       INNER JOIN reviews ON reviews.username = subquery1.username
                   GROUP BY
                       subquery1.username) AS subquery2"""

mean_ratings = pd.read_sql(query_5, con=engine)
print(mean_ratings)

   avg_review_cnt
0       24.333333


<span style="color: black; font-family: Arial; font-size: 20px;">
Usuários que classificaram mais do que 50 livros deram em média 24 avaliações.